## Name generator bot

In [ ]:
import random
import json
import os
from telegram import InlineKeyboardButton, InlineKeyboardMarkup, ForceReply, Update
from telegram.ext import (
    Updater, 
    CommandHandler, 
    MessageHandler, 
    Filters, 
    CallbackContext, 
    CallbackQueryHandler, 
    ConversationHandler,
)

In [ ]:
def start(update: Update, context: CallbackContext) -> None:
        update.message.reply_text(
            "A good beginning is most important\n"
            "Type /add to add a name\n"
            "Type /clear to clear current name collection\n"
            "Type /random to show random name\n"
        )
        
        
class Names:

    # state for ConversationHandler
    DEFAULT, FIRST, LAST = range(3)
    # callback data
    FIRST_NAME, LAST_NAME, NOTHING = range(3)

    def __init__(self, dispatcher):
        conv_handler = ConversationHandler(
            entry_points=[CommandHandler("add", self.add)],
            states={
                self.DEFAULT: [
                    CallbackQueryHandler(self.add_first_name, pattern='^' + str(self.FIRST_NAME) + '$'),
                    CallbackQueryHandler(self.add_last_name, pattern='^' + str(self.LAST_NAME) + '$'),
                ],
                self.FIRST: [MessageHandler(Filters.text & ~Filters.command, self.added_first_name)],
                self.LAST: [MessageHandler(Filters.text & ~Filters.command, self.added_last_name)],
            },
            fallbacks=[CallbackQueryHandler(self.end, pattern='^' + str(self.NOTHING) + '$')],
        )
        dispatcher.add_handler(conv_handler)
        dispatcher.add_handler(CommandHandler("clear", self.clear))
        dispatcher.add_handler(CommandHandler("random", self.random))

    # side methods
    
    def write_name_to_file(self, name: str, type_: str) -> None:
        with open("data/file.json", "r") as file:
            data = json.load(file)
        data[type_].append(name)
        with open("data/file.json", "w") as file:
            json.dump(data, file)
        
    # bot methods
    
    def add(self, update: Update, context: CallbackContext) -> int:
        keyboard = [
            [
                InlineKeyboardButton("First name", callback_data=str(self.FIRST_NAME)),
                InlineKeyboardButton("Last name", callback_data=str(self.LAST_NAME)),
            ],
            [InlineKeyboardButton("Nothing", callback_data=str(self.NOTHING))],
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)
        update.message.reply_text(
            "What do you want to add?",
            reply_markup=reply_markup
        )
        return self.DEFAULT

    def add_first_name(self, update: Update, context: CallbackContext) -> int:
        query = update.callback_query
        query.answer()
        query.edit_message_text("Enter first name", reply_markup=InlineKeyboardMarkup([]))
        return self.FIRST

    def add_last_name(self, update: Update, context: CallbackContext) -> int:
        query = update.callback_query
        query.answer()
        query.edit_message_text("Enter last name", reply_markup=InlineKeyboardMarkup([]))
        return self.LAST

    def added_first_name(self, update: Update, context: CallbackContext) -> int:
        name = update.message.text
        update.message.reply_text(f'Added first name {name}')
        update.message.delete()
        self.write_name_to_file(name, 'first_name')
        return ConversationHandler.END

    def added_last_name(self, update: Update, context: CallbackContext) -> int:
        name = update.message.text
        update.message.reply_text(f'Added last name {name}')
        update.message.delete()
        self.write_name_to_file(name, 'last_name')
        return ConversationHandler.END

    def end(self, update: Update, context: CallbackContext) -> int:
        query = update.callback_query
        query.answer()
        query.delete_message()
        return ConversationHandler.END

    def clear(self, update: Update, context: CallbackContext) -> None:
        data = {'first_name': [], 'last_name': []}
        with open("data/file.json", "w",) as file:
            json.dump(data, file)
        update.message.reply_text('All names were banished... forever')

    def random(self, update: Update, context: CallbackContext) -> None:
        with open("data/file.json", "r") as file:
            data = json.load(file)
        if data['first_name'] != [] and data['last_name'] != []:
            first_name = random.choice(data['first_name'])
            last_name = random.choice(data['last_name'])
            update.message.reply_text(f'Here is your absolutely random name: {first_name} {last_name}')
        else:
            update.message.reply_text('No names, sorry')
            
            
def main():
    updater = Updater(TOKEN)
    dispatcher = updater.dispatcher
    
    dispatcher.add_handler(CommandHandler("start", start))
    
    names = Names(dispatcher)
    
    updater.start_polling()
    updater.idle()

In [ ]:
main()